In [ ]:

from actions import *
import re
from time import sleep
import os
from dotenv import load_dotenv
import requests


In [2]:
if not load_dotenv():
    raise Exception("env vars not loaded")

API_KEY = os.getenv("API_KEY")

In [ ]:
url = "https://api.x.ai/v1/chat/completions"

headers = {
    "Authorization": "Bearer ",
    "Content-Type": "application/json"
}

data = {
    "messages": [
        {
            "role": "system",
            "content": "You are a test assistant."
        },
        {
            "role": "user",
            "content": "Tell me a short, edgy joke"
        }
    ],
    "model": "grok-beta",
    "stream": False,
    "temperature": 1
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
response.text

400


'{"code":"Client specified an invalid argument","error":"Invalid Authorization header \'Bearer\'. Specify your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY). You can obtain an API key from https://console.x.ai"}'

In [4]:
def grok(messages: list[dict]) -> str:
    global API_KEY
    if not(isinstance(messages, list)):
        raise BaseException('Use message list when calling grok()')
    
    url = "https://api.x.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "messages": messages,
        "model": "grok-beta",
        "stream": False,
        "temperature": 1
    }

    response = requests.post(url, headers=headers, json=data)
    
    return response.json()["choices"][0]["message"]["content"]

In [8]:
grok([{"role":"system", "content": "hello!"}])

'Hello! How can I assist you today?'

In [4]:
class Agent():
    
    def __init__(self) -> None:
        self.messages = []
        return
    
    def init_thinking_behaviour(self):
        behaviour = """
            You are an expert survivalist. You are able to suggest step by step 
            how to orient yourself in nature, build primitive tools and survive 
            or escape. You are logical thinking.
            
            I'm asking you in a loop. Each time I ask, I want you to suggest
            in plain english, what the logical next steps should be.
            
            You can only use decrete actions, here is the list:
            collect wood, start fire, build shelter, hunt animal, fish, purify water,
            gather plants, make weapon, set trap, create signal, navigate nature,
            find water, make container, make rope, make insulation, make tools, 
            make stretcher, find high ground, scavenge materials, track animal, 
            climb_tree, spot_village_from_highpoint
            
            Decide which one to use and answer which one. Make it look like python function. Example:
            - decision: colled wood, resposne: "action: collect_wood()"
            - decision: make rope, resposne: "action: make_rope()"
        
        """.strip()
        self.messages.append({"role": "system", "content": behaviour})
    
    def ask(self, prompt=''):
        if prompt:
            self.messages.append({"role": "user", "content": prompt})
            answer = grok(self.messages)
            self.messages.append({"role": "user", "content": answer})
            
        else:
            last_message = self.messages[-1]["content"]
            if "<STOP>" in last_message:
                return "<STOP>"
            elif "action:" in last_message:
                func = re.findall(r'action:\s*(\w+)\(\)', last_message)[0]
                answer = eval(func+"()")
                self.messages.append({"role": "user", "content": answer})
            else:
                self.messages.append({"role": "user", "content": "That is done. Now, on to next logical thing."})
                answer = grok(self.messages)
                self.messages.append({"role": "user", "content": answer})
        
        return "<CONTINUE>"
                
                

In [5]:
smarty = Agent()
smarty.init_thinking_behaviour()

In [6]:
iterations = 20

smarty.ask("You are trapped in Norway forest. No rules. Stay alive and find village asap")
i = 0
while i < iterations:
    i += 1
    status = smarty.ask()
    if status == "<STOP>":
        break
    sleep(1)
    print(smarty.messages[-1])
    
    

{'role': 'user', 'content': 'Navigated using natural indicators.'}
{'role': 'user', 'content': 'To establish a base camp which will be useful in your survival, next step is:\n\n```python\naction: find_high_ground()\n```'}
{'role': 'user', 'content': 'Located high ground for visibility.'}
{'role': 'user', 'content': "To assess your surroundings and plan your next move, here's what to do:\n\n```python\naction: spot_village_from_highpoint()\n```"}
{'role': 'user', 'content': 'You found a village! <STOP>'}


In [7]:
with open("./out.txt", "w") as f:
    f.truncate()
    f.seek(0)
    for i in smarty.messages:
        for k, v in i.items():
            f.write(f"{k}\n\t{v}")
            f.write("\n\n")
            